### Preprocessing 

In [3]:
import os
import xml.etree.ElementTree as ET
import cv2

def convert_to_yolo_format(bbox, img_size, class_id=0):
    """
    Convert bbox from Pascal VOC format [xmin, ymin, xmax, ymax] to YOLO format.
    """
    dw = 1. / img_size[0]
    dh = 1. / img_size[1]
    x = (bbox[0] + bbox[2]) / 2.0
    y = (bbox[1] + bbox[3]) / 2.0
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}"

def parse_annotation(xml_file, image_folder):
    """
    Parse the XML file to get the bounding box coordinates and write them in YOLO format.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    filename = root.find('filename').text
    image_path = os.path.join(image_folder, filename)
    img_size = cv2.imread(image_path).shape
    
    boxes = []
    for obj in root.iter('object'):
        class_id = 0  # TODO: Set the correct class ID if you have multiple classes
        xmlbox = obj.find('bndbox')
        bbox = (
            int(xmlbox.find('xmin').text),
            int(xmlbox.find('ymin').text),
            int(xmlbox.find('xmax').text),
            int(xmlbox.find('ymax').text),
        )
        boxes.append(convert_to_yolo_format(bbox, img_size, class_id))
    return filename, boxes

def save_yolo_annotations(annotations_path, image_folder, output_path):
    """
    Convert and save the annotations in YOLO format.
    """
    for xml_file in os.listdir(annotations_path):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(annotations_path, xml_file)
            filename, boxes = parse_annotation(xml_path, image_folder)
            
            # Remove image extension and add .txt extension
            base_filename = os.path.splitext(filename)[0]
            output_txt_path = os.path.join(output_path, f"{base_filename}.txt")
            
            # Write the YOLO formatted bounding boxes to a .txt file
            with open(output_txt_path, 'w') as file:
                for box in boxes:
                    file.write(box + "\n")

# Paths to your dataset
annotations_path = 'archive/annotations'
image_folder = 'archive/images'
output_path = 'archive/processed_images'

# Convert and save annotations in YOLO format
save_yolo_annotations(annotations_path, image_folder, output_path)


ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
import easyocr

In [70]:

def get_labels(image_directory):
    images, labels, cropped_plates = [], [], []
    reader = Reader(['en'])

    #counter = 1

    for filename in os.listdir(image_directory):
        #print(counter)
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            print(filename)
            img_path = os.path.join(image_directory, filename)
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            bfilter = cv2.bilateralFilter(gray, 11, 11, 17)
            edged = cv2.Canny(bfilter, 30, 200)

            keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(keypoints)
            contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]

            location = None
            for contour in contours:
            # cv2.approxPolyDP returns a resampled contour, so this will still return a set of (x, y) points
                approx = cv2.approxPolyDP(contour, 10, True)
                if len(approx) == 4:
                    location = approx
                    break

            try:
                mask = np.zeros(gray.shape, np.uint8)
                new_image = cv2.drawContours(mask, [location], 0, 255, -1)
                new_image = cv2.bitwise_and(img, img, mask = mask)
            except:
                continue

            (x, y) = np.where(mask == 255)
            (x1, y1) = (np.min(x), np.min(y))
            (x2, y2) = (np.max(x), np.max(y))
            # Adding Buffer
            cropped_image = gray[x1:x2+3, y1:y2+3]
            #plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

            result = reader.readtext(cropped_image)
            if len(result) != 0:
                images.append(img)
                cropped_plates.append(cropped_image)
                labels.append(result[0][1])
        #counter += 1

    return images, labels, cropped_plates

In [71]:
image_directory = 'archive/images'

images, labels, cropped_plates = get_labels(image_directory)

Cars207.png
Cars213.png
Cars53.png
Cars47.png
Cars90.png
Cars159.png
Cars84.png
Cars171.png
Cars165.png
Cars398.png
Cars429.png
Cars373.png
Cars415.png
Cars401.png
Cars367.png
Cars400.png
Cars366.png
Cars372.png
Cars414.png
Cars428.png
Cars399.png
Cars164.png
Cars170.png
Cars85.png
Cars158.png
Cars91.png
Cars46.png
Cars52.png
Cars212.png
Cars206.png
Cars238.png
Cars210.png
Cars204.png
Cars199.png
Cars44.png
Cars50.png
Cars78.png
Cars87.png
Cars93.png
Cars166.png
Cars172.png
Cars358.png
Cars364.png
Cars402.png
Cars416.png
Cars370.png
Cars417.png
Cars371.png
Cars365.png
Cars403.png
Cars359.png
Cars173.png
Cars167.png
Cars92.png
Cars86.png
Cars79.png
Cars51.png
Cars45.png
Cars198.png
Cars205.png
Cars211.png
Cars239.png
Cars215.png
Cars201.png
Cars229.png
Cars69.png
Cars41.png
Cars55.png
Cars188.png
Cars163.png
Cars177.png
Cars82.png
Cars96.png
Cars407.png
Cars361.png
Cars375.png
Cars413.png
Cars349.png
Cars348.png
Cars374.png
Cars412.png
Cars406.png
Cars360.png
Cars97.png
Cars83.png
Cars1

In [73]:
len(labels)

150

In [64]:

img = cv2.imread('archive/images/Cars213.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [65]:
bfilter = cv2.bilateralFilter(gray, 11, 11, 17)
edged = cv2.Canny(bfilter, 30, 200)
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

In [66]:
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]

location = None
for contour in contours:
  # cv2.approxPolyDP returns a resampled contour, so this will still return a set of (x, y) points
  approx = cv2.approxPolyDP(contour, 10, True)
  if len(approx) == 4:
    location = approx
    break

In [67]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0, 255, -1)
new_image = cv2.bitwise_and(img, img, mask = mask)
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

In [68]:
(x, y) = np.where(mask == 255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
# Adding Buffer
cropped_image = gray[x1:x2+3, y1:y2+3]
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

In [69]:
reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
print(result)

[]


In [55]:
print(result[0][1])

PGeNN112
